In [2]:
import pandas as pd
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import bigrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearnex import patch_sklearn
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils.class_weight import compute_class_weight
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from imblearn.over_sampling import RandomOverSampler
patch_sklearn()
nltk.download('stopwords')
nltk.download('punkt')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lakbo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lakbo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#preprocessing
"""movie_details = pd.read_json('../data/IMDB_movie_details.json', lines = True)
imdb_reviews = pd.read_json('../data/IMDB_reviews.json', lines = True)"""

"movie_details = pd.read_json('../data/IMDB_movie_details.json', lines = True)\nimdb_reviews = pd.read_json('../data/IMDB_reviews.json', lines = True)"

In [4]:
#check for na
"""no_summary = movie_details[movie_details['plot_summary'].isna()]
no_summary"""

"no_summary = movie_details[movie_details['plot_summary'].isna()]\nno_summary"

In [5]:
#check for na
"""no_synopsis = movie_details[movie_details['plot_synopsis'].isna()]
no_synopsis"""

"no_synopsis = movie_details[movie_details['plot_synopsis'].isna()]\nno_synopsis"

In [6]:
#check for na
"""no_review = imdb_reviews[imdb_reviews['review_text'].isna()]
no_review"""

"no_review = imdb_reviews[imdb_reviews['review_text'].isna()]\nno_review"

In [7]:
#functions for preprocessing
"""def lowercase(dataframe,columnname):
    lowercase_dataframe = dataframe[columnname].apply(lambda x: x.lower())
    return lowercase_dataframe

def remove_special_characters(dataframe,columnname):
    dataframe_no_special_characters = dataframe[columnname].replace(r'[^A-Za-z0-9 ]+', '', regex=True)
    return dataframe_no_special_characters

def tokenize_words(dataframe,columnname):
    dataframe_tokenized_texts= dataframe[columnname].apply(lambda x: word_tokenize(x) )
    return dataframe_tokenized_texts

def remove_stop_words(dataframe,columnname):
    stop = stopwords.words('english')
    dataframe_no_stop_words= dataframe[columnname].apply(lambda x: [item for item in x if item not in stop])
    return dataframe_no_stop_words
"""

"def lowercase(dataframe,columnname):\n    lowercase_dataframe = dataframe[columnname].apply(lambda x: x.lower())\n    return lowercase_dataframe\n\ndef remove_special_characters(dataframe,columnname):\n    dataframe_no_special_characters = dataframe[columnname].replace(r'[^A-Za-z0-9 ]+', '', regex=True)\n    return dataframe_no_special_characters\n\ndef tokenize_words(dataframe,columnname):\n    dataframe_tokenized_texts= dataframe[columnname].apply(lambda x: word_tokenize(x) )\n    return dataframe_tokenized_texts\n\ndef remove_stop_words(dataframe,columnname):\n    stop = stopwords.words('english')\n    dataframe_no_stop_words= dataframe[columnname].apply(lambda x: [item for item in x if item not in stop])\n    return dataframe_no_stop_words\n"

In [8]:
#Processing of data
"""imdb_reviews = imdb_reviews.head()
imdb_reviews['review_text'] = lowercase(imdb_reviews,'review_text')
imdb_reviews['review_text'] = remove_special_characters(imdb_reviews,'review_text')
imdb_reviews['review_text'] = tokenize_words(imdb_reviews,'review_text')
imdb_reviews['review_text'] = remove_stop_words(imdb_reviews,'review_text')
imdb_reviews"""

"imdb_reviews = imdb_reviews.head()\nimdb_reviews['review_text'] = lowercase(imdb_reviews,'review_text')\nimdb_reviews['review_text'] = remove_special_characters(imdb_reviews,'review_text')\nimdb_reviews['review_text'] = tokenize_words(imdb_reviews,'review_text')\nimdb_reviews['review_text'] = remove_stop_words(imdb_reviews,'review_text')\nimdb_reviews"

In [9]:
#change to lowercase
"""imdb_reviews['review_text'] = lowercase(imdb_reviews,'review_text')
#remove special characters
imdb_reviews['review_text'] = remove_special_characters(imdb_reviews,'review_text')
#tokenize text 15min
imdb_reviews['review_text'] = tokenize_words(imdb_reviews,'review_text')
#remove stop words 8min
imdb_reviews['review_text'] = remove_stop_words(imdb_reviews,'review_text')
imdb_reviews['is_spoiler'] = imdb_reviews['is_spoiler'].map({True: 1, False: 0})
imdb_reviews.head()"""

"imdb_reviews['review_text'] = lowercase(imdb_reviews,'review_text')\n#remove special characters\nimdb_reviews['review_text'] = remove_special_characters(imdb_reviews,'review_text')\n#tokenize text 15min\nimdb_reviews['review_text'] = tokenize_words(imdb_reviews,'review_text')\n#remove stop words 8min\nimdb_reviews['review_text'] = remove_stop_words(imdb_reviews,'review_text')\nimdb_reviews['is_spoiler'] = imdb_reviews['is_spoiler'].map({True: 1, False: 0})\nimdb_reviews.head()"

In [10]:
#bigrams and bigram text
"""imdb_reviews['bigrams'] = imdb_reviews['review_text'].apply(lambda x: list(bigrams(x)))
imdb_reviews['bigram_text'] = imdb_reviews['bigrams'].apply(lambda x: ' '.join([' '.join(bigram) for bigram in x]))"""

"imdb_reviews['bigrams'] = imdb_reviews['review_text'].apply(lambda x: list(bigrams(x)))\nimdb_reviews['bigram_text'] = imdb_reviews['bigrams'].apply(lambda x: ' '.join([' '.join(bigram) for bigram in x]))"

In [11]:
#From csv file with bigrams text
imdb_reviews_bigrams = pd.read_csv('../data/imdb_reviews_nostop_bigrams.csv')

In [12]:
imdb_reviews_bigrams.head()

,Unnamed: 0,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary,bigrams,bigram_text
0,0,10 February 2006,tt0111161,ur1898687,1,"['oscar', 'year', 'shawshank', 'redemption', '...",10,A classic piece of unforgettable film-making.,"[('oscar', 'year'), ('year', 'shawshank'), ('s...",oscar year year shawshank shawshank redemption...
1,1,6 September 2000,tt0111161,ur0842118,1,"['shawshank', 'redemption', 'without', 'doubt'...",10,Simply amazing. The best film of the 90's.,"[('shawshank', 'redemption'), ('redemption', '...",shawshank redemption redemption without withou...
2,2,3 August 2001,tt0111161,ur1285640,1,"['believe', 'film', 'best', 'story', 'ever', '...",8,The best story ever told on film,"[('believe', 'film'), ('film', 'best'), ('best...",believe film film best best story story ever e...
3,3,1 September 2002,tt0111161,ur1003471,1,"['yes', 'spoilers', 'herethis', 'film', 'emoti...",10,Busy dying or busy living?,"[('yes', 'spoilers'), ('spoilers', 'herethis')...",yes spoilers spoilers herethis herethis film f...
4,4,20 May 2004,tt0111161,ur0226855,1,"['heart', 'extraordinary', 'movie', 'brilliant...",8,"Great story, wondrously told and acted","[('heart', 'extraordinary'), ('extraordinary',...",heart extraordinary extraordinary movie movie ...


In [13]:
#check for na
imdb_reviews_bigrams[imdb_reviews_bigrams['bigram_text'].isna()]

,Unnamed: 0,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary,bigrams,bigram_text
30552,30552,24 December 2017,tt0120586,ur64553360,0,['minxfyicbcfrxesydrfvbhjnmkjiuytfrdesdcfvghju...,10,the Best,[],NaN
33433,33433,27 December 2017,tt0027977,ur83694676,1,['nice'],9,Very good,[],NaN
269257,269257,20 December 2017,tt2239822,ur83454181,0,['greatgreatgreatgreatgreatgreatgreatgreatgrea...,9,great,[],NaN
487228,487228,15 October 1998,tt0102685,ur0111020,0,['headacheinducing'],1,Two words:,[],NaN


In [14]:
#remove na data
imdb_reviews_bigrams = imdb_reviews_bigrams.dropna(subset=['bigram_text'])
imdb_reviews_bigrams.head()

,Unnamed: 0,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary,bigrams,bigram_text
0,0,10 February 2006,tt0111161,ur1898687,1,"['oscar', 'year', 'shawshank', 'redemption', '...",10,A classic piece of unforgettable film-making.,"[('oscar', 'year'), ('year', 'shawshank'), ('s...",oscar year year shawshank shawshank redemption...
1,1,6 September 2000,tt0111161,ur0842118,1,"['shawshank', 'redemption', 'without', 'doubt'...",10,Simply amazing. The best film of the 90's.,"[('shawshank', 'redemption'), ('redemption', '...",shawshank redemption redemption without withou...
2,2,3 August 2001,tt0111161,ur1285640,1,"['believe', 'film', 'best', 'story', 'ever', '...",8,The best story ever told on film,"[('believe', 'film'), ('film', 'best'), ('best...",believe film film best best story story ever e...
3,3,1 September 2002,tt0111161,ur1003471,1,"['yes', 'spoilers', 'herethis', 'film', 'emoti...",10,Busy dying or busy living?,"[('yes', 'spoilers'), ('spoilers', 'herethis')...",yes spoilers spoilers herethis herethis film f...
4,4,20 May 2004,tt0111161,ur0226855,1,"['heart', 'extraordinary', 'movie', 'brilliant...",8,"Great story, wondrously told and acted","[('heart', 'extraordinary'), ('extraordinary',...",heart extraordinary extraordinary movie movie ...


In [15]:
#vectorize bigram text
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(imdb_reviews_bigrams['bigram_text'])

In [50]:
imdb_reviews_bigrams

,Unnamed: 0,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary,bigrams,bigram_text
0,0,10 February 2006,tt0111161,ur1898687,1,"['oscar', 'year', 'shawshank', 'redemption', '...",10,A classic piece of unforgettable film-making.,"[('oscar', 'year'), ('year', 'shawshank'), ('s...",oscar year year shawshank shawshank redemption...
1,1,6 September 2000,tt0111161,ur0842118,1,"['shawshank', 'redemption', 'without', 'doubt'...",10,Simply amazing. The best film of the 90's.,"[('shawshank', 'redemption'), ('redemption', '...",shawshank redemption redemption without withou...
2,2,3 August 2001,tt0111161,ur1285640,1,"['believe', 'film', 'best', 'story', 'ever', '...",8,The best story ever told on film,"[('believe', 'film'), ('film', 'best'), ('best...",believe film film best best story story ever e...
3,3,1 September 2002,tt0111161,ur1003471,1,"['yes', 'spoilers', 'herethis', 'film', 'emoti...",10,Busy dying or busy living?,"[('yes', 'spoilers'), ('spoilers', 'herethis')...",yes spoilers spoilers herethis herethis film f...
4,4,20 May 2004,tt0111161,ur0226855,1,"['heart', 'extraordinary', 'movie', 'brilliant...",8,"Great story, wondrously told and acted","[('heart', 'extraordinary'), ('extraordinary',...",heart extraordinary extraordinary movie movie ...
...,...,...,...,...,...,...,...,...,...,...
573908,573908,8 August 1999,tt0139239,ur0100166,0,"['go', 'wise', 'fast', 'pure', 'entertainment'...",10,The best teen movie of the nineties,"[('go', 'wise'), ('wise', 'fast'), ('fast', 'p...",go wise wise fast fast pure pure entertainment...
573909,573909,31 July 1999,tt0139239,ur0021767,0,"['well', 'shall', 'say', 'ones', 'fun', 'rate'...",9,Go - see the movie,"[('well', 'shall'), ('shall', 'say'), ('say', ...",well shall shall say say ones ones fun fun rat...
573910,573910,20 July 1999,tt0139239,ur0392750,0,"['go', 'best', 'movie', 'ever', 'seen', 'ive',...",10,It's the best movie I've ever seen,"[('go', 'best'), ('best', 'movie'), ('movie', ...",go best best movie movie ever ever seen seen i...
573911,573911,11 June 1999,tt0139239,ur0349105,0,"['call', '1999', 'teenage', 'version', 'pulp',...",3,Haven't we seen this before?,"[('call', '1999'), ('1999', 'teenage'), ('teen...",call 1999 1999 teenage teenage version version...


In [8]:
# Split the data into training and testing sets
X = matrix
y = imdb_reviews_bigrams['is_spoiler']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize SelectKBest with chi-squared as the score function
k_best_selector = SelectKBest(score_func=chi2, k=100) #can change k accordingly

X_train_selected = k_best_selector.fit_transform(X_train, y_train)
X_test_selected = k_best_selector.transform(X_test) 

In [ ]:
#logistic regression model
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_selected, y_train)

# Predict using the trained model on the selected test set
y_pred = log_model.predict(X_test_selected)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate F1 score
f1score = f1_score(y_test, y_pred)
print(f"F1 Score: {f1score:.2f}")

Accuracy: 0.76
Confusion Matrix:
[[81065  3584]
 [24075  6058]]
F1 Score: 0.30


In [ ]:
#SVM model
# Initialize and train SVM with selected features
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_selected, y_train)

# Predict using the trained SVM model on the selected test set
y_pred_svm = svm_model.predict(X_test_selected)

# Calculate accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {accuracy_svm:.2f}")

# Compute confusion matrix
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
print("SVM Confusion Matrix:")
print(conf_matrix_svm)

# Calculate F1 score
f1score_svm = f1_score(y_test, y_pred_svm)
print(f"SVM F1 Score: {f1score_svm:.2f}")

SVM Accuracy: 0.75
SVM Confusion Matrix:
[[82606  2043]
 [26349  3784]]
SVM F1 Score: 0.21


In [ ]:
#randomforest model
# Initialize Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust n_estimators and other hyperparameters

# Train the Random Forest model
rf_model.fit(X_train_selected, y_train)

# Predict using the trained Random Forest model on the test set
X_test_selected = k_best_selector.transform(X_test)  # Transform the test set using the same feature selection
y_pred_rf = rf_model.predict(X_test_selected)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.2f}")

# Compute confusion matrix
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("Random Forest Confusion Matrix:")
print(conf_matrix_rf)

# Calculate F1 score
f1score_rf = f1_score(y_test, y_pred_rf)
print(f"Random Forest F1 Score: {f1score_rf:.2f}")

Random Forest Accuracy: 0.76
Random Forest Confusion Matrix:
[[80980  3669]
 [23900  6233]]
Random Forest F1 Score: 0.31


Accuracy:
Accuracy is the proportion of correctly classified instances. While Random Forest, logistic regression and SVM have similar accuracies (around 75-76%), it's important to consider other metrics for a more complete picture.

F1 Score:
F1 score is the harmonic mean of precision and recall. It's a good metric when there's an imbalance in the class distribution. In your case, Random Forest has a slightly higher F1 score (0.31) compared to SVM (0.21) and logistic regression (0.30)

Conclusion:
Logistic Regression and Random Forest perform similarly, with a slight advantage in F1 score for Random Forest.
SVM, while having a comparable accuracy, lags behind in terms of F1 score, indicating a trade-off between precision and recall.
Consider the specific goals of your application. If a balanced trade-off between precision and recall is crucial, Random Forest may be a reasonable choice. If interpretability is a priority, Logistic Regression might be preferred.
Further model tuning, feature engineering, or exploring advanced techniques (such as deep learning) could potentially improve the performance of your models.


We will be focusing on lstm as it is effective at capturing complex relationships in sequential data.
It requires alot of data which in this case we have.

In [5]:
#lstm (1/2)
X = imdb_reviews_bigrams['bigram_text']
y = imdb_reviews_bigrams['is_spoiler']

#split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#vectorize
vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features based on your data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Feature selection using SelectKBest on training set
k_best_selector = SelectKBest(score_func=chi2, k=100)  # You can adjust k based on your data
X_train_selected = k_best_selector.fit_transform(X_train_tfidf, y_train)
X_test_selected = k_best_selector.transform(X_test_tfidf)

# Scale the data
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Convert labels to binary format
y_train_binary = y_train.astype('int')
y_test_binary = y_test.astype('int')

# Tokenize and pad sequences
max_words = 10000  # You can adjust based on your vocabulary size
max_len = 50 # You can adjust based on the desired sequence length

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post', truncating='post')

In [61]:
#lstm model(2/2)
# Build the LSTM model
embedding_dim = 50  # You can adjust based on your choice
lstm_units = 100  # You can adjust based on your choice

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(units=lstm_units))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(X_train_padded, y_train_binary, epochs=10, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model on the test set
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_binary, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test_binary, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate F1 score
f1score = f1_score(y_test_binary, y_pred)
print(f"F1 Score: {f1score:.4f}")

Epoch 1/10
6457/6457 [==============================] - 385s 59ms/step - loss: 0.5558 - accuracy: 0.7446 - val_loss: 0.5516 - val_accuracy: 0.7468
Epoch 2/10
6457/6457 [==============================] - 352s 55ms/step - loss: 0.5397 - accuracy: 0.7489 - val_loss: 0.5461 - val_accuracy: 0.7473
Epoch 3/10
6457/6457 [==============================] - 343s 53ms/step - loss: 0.5309 - accuracy: 0.7508 - val_loss: 0.5484 - val_accuracy: 0.7467
Epoch 4/10
6457/6457 [==============================] - 336s 52ms/step - loss: 0.5213 - accuracy: 0.7545 - val_loss: 0.5544 - val_accuracy: 0.7387
Epoch 5/10
3587/3587 [==============================] - 36s 10ms/step
Test Accuracy: 0.7487
Confusion Matrix:
[[83352  1297]
 [27552  2581]]
F1 Score: 0.1518


The reason for a high accuracy but a low F1 score in the context of the LSTM model could be attributed to class imbalance. F1 score takes into account both precision and recall, and it's particularly sensitive to false positives and false negatives.

In [62]:
#check for class imbalance
count_classes = imdb_reviews_bigrams['is_spoiler'].value_counts()
print(count_classes)

is_spoiler
0    422986
1    150923
Name: count, dtype: int64


In [16]:
#lstm addressing imbalance class(1/2)
X = imdb_reviews_bigrams['bigram_text']
y = imdb_reviews_bigrams['is_spoiler']

#split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

#vectorize
vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features based on your data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Feature selection using SelectKBest on training set
k_best_selector = SelectKBest(score_func=chi2, k=100)  # You can adjust k based on your data
X_train_selected = k_best_selector.fit_transform(X_train_tfidf, y_train)
X_test_selected = k_best_selector.transform(X_test_tfidf)

# Scale the data
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Convert labels to binary format
y_train_binary = y_train.astype('int')
y_test_binary = y_test.astype('int')

# Tokenize and pad sequences
max_words = 10000  # You can adjust based on your vocabulary size
max_len = 50 # You can adjust based on the desired sequence length

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post', truncating='post')

In [65]:
#lstm addressing imbalance class(2/2)
# Build the LSTM model
embedding_dim = 50  # You can adjust based on your choice
lstm_units = 100  # You can adjust based on your choice

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(units=lstm_units))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with class weights
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with class weights
history = model.fit(X_train_padded, y_train_binary, epochs=10, batch_size=64, validation_split=0.1, callbacks=[early_stopping], class_weight=class_weights_dict)

# Evaluate the model on the test set
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_binary, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test_binary, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate F1 score
f1score = f1_score(y_test_binary, y_pred)
print(f"F1 Score: {f1score:.4f}")

Epoch 1/10
6457/6457 [==============================] - 614s 95ms/step - loss: 0.6620 - accuracy: 0.5864 - val_loss: 0.6568 - val_accuracy: 0.5848
Epoch 2/10
6457/6457 [==============================] - 391s 61ms/step - loss: 0.6418 - accuracy: 0.6142 - val_loss: 0.6576 - val_accuracy: 0.5940
Epoch 3/10
6457/6457 [==============================] - 411s 64ms/step - loss: 0.6216 - accuracy: 0.6350 - val_loss: 0.6553 - val_accuracy: 0.5982
Epoch 4/10
6457/6457 [==============================] - 474s 73ms/step - loss: 0.5970 - accuracy: 0.6590 - val_loss: 0.6625 - val_accuracy: 0.6012
Epoch 5/10
6457/6457 [==============================] - 385s 60ms/step - loss: 0.5716 - accuracy: 0.6788 - val_loss: 0.6838 - val_accuracy: 0.5988
Epoch 6/10
3587/3587 [==============================] - 32s 9ms/step
Test Accuracy: 0.5986
Confusion Matrix:
[[51370 33279]
 [12789 17344]]
F1 Score: 0.4295


Given the context of detecting movie spoilers, where the classes are imbalanced, optimizing for F1 score might be more relevant. F1 score considers both precision and recall, providing a better balance, especially when false positives and false negatives carry different costs. Drop in accuracy is accompanied by an improvement in F1 score and better performance on the minority class (spoilers in this case), it could be considered a worthwhile trade-off. 

In [86]:
#Improving accuracy for lstm after addressing class imbalance
# Build the Bidirectional LSTM model
embedding_dim = 50
lstm_units = 100

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(units=lstm_units)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with class weights
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with class weights
history = model.fit(X_train_padded, y_train_binary, epochs=10, batch_size=64, validation_split=0.1, callbacks=[early_stopping], class_weight=class_weights_dict)

# Evaluate the model on the test set
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_binary, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test_binary, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate F1 score
f1score = f1_score(y_test_binary, y_pred)
print(f"F1 Score: {f1score:.4f}")

Epoch 1/10
6457/6457 [==============================] - 1571s 243ms/step - loss: 0.6593 - accuracy: 0.5967 - val_loss: 0.6652 - val_accuracy: 0.5777
Epoch 2/10
6457/6457 [==============================] - 2690s 417ms/step - loss: 0.6374 - accuracy: 0.6260 - val_loss: 0.6526 - val_accuracy: 0.5985
Epoch 3/10
6457/6457 [==============================] - 2928s 453ms/step - loss: 0.6148 - accuracy: 0.6502 - val_loss: 0.6537 - val_accuracy: 0.5963
Epoch 4/10
6457/6457 [==============================] - 2524s 391ms/step - loss: 0.5895 - accuracy: 0.6737 - val_loss: 0.6945 - val_accuracy: 0.5693
Epoch 5/10
3587/3587 [==============================] - 159s 44ms/step
Test Accuracy: 0.5980
Confusion Matrix:
[[50507 34142]
 [12000 18133]]
F1 Score: 0.4401


 Bidirectional LSTMs come with additional hyperparameters, such as the number of units in each direction, dropout rates, etc. If these hyperparameters are not tuned properly, they might negatively impact the performance of the bidirectional model.

In [98]:
#fine tuning the hyperparameters of the models above
#Improving accuracy for lstm after addressing class imbalance
# Build the Bidirectional LSTM model
embedding_dim = 64 #50 -> 64
lstm_units = 128 #100 -> 128 

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(units=lstm_units)))
model.add(Dropout(0.4)) #0.5 -> 0.4
model.add(Dense(1, activation='sigmoid'))

# Compile the model with class weights
optimizer = Adam(learning_rate=0.0005) #decrease learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with class weights
history = model.fit(X_train_padded, y_train_binary, epochs=15, batch_size=128, validation_split=0.15, callbacks=[early_stopping], class_weight=class_weights_dict) #epochs from 10, batch size from 64 and validation from 0.1

# Evaluate the model on the test set
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_binary, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test_binary, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate F1 score
f1score = f1_score(y_test_binary, y_pred)
print(f"F1 Score: {f1score:.4f}")

Epoch 1/15
3049/3049 [==============================] - 4036s 1s/step - loss: 0.6587 - accuracy: 0.5965 - val_loss: 0.6615 - val_accuracy: 0.5887
Epoch 2/15
3049/3049 [==============================] - 4469s 1s/step - loss: 0.6389 - accuracy: 0.6279 - val_loss: 0.6527 - val_accuracy: 0.5945
Epoch 3/15
3049/3049 [==============================] - 3064s 1s/step - loss: 0.6200 - accuracy: 0.6470 - val_loss: 0.6528 - val_accuracy: 0.6010
Epoch 4/15
3049/3049 [==============================] - 3537s 1s/step - loss: 0.5983 - accuracy: 0.6653 - val_loss: 0.6484 - val_accuracy: 0.6174
Epoch 5/15
3049/3049 [==============================] - 4192s 1s/step - loss: 0.5757 - accuracy: 0.6825 - val_loss: 0.6532 - val_accuracy: 0.6212
Epoch 6/15
3049/3049 [==============================] - 4204s 1s/step - loss: 0.5519 - accuracy: 0.6995 - val_loss: 0.7088 - val_accuracy: 0.5841
Epoch 7/15
3587/3587 [==============================] - 298s 83ms/step
Test Accuracy: 0.6185
Confusion Matrix:
[[55507 29142

In [11]:
#fine tuning the hyperparameters of the models
#Improving accuracy for lstm after addressing class imbalance
# Build the Bidirectional LSTM model
embedding_dim = 128 #64 -> 128
lstm_units = 256 #128 -> 256

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(units=lstm_units)))
model.add(Dropout(0.4)) #0.5 -> 0.4
model.add(Dense(1, activation='sigmoid'))

# Compile the model with class weights
optimizer = Adam(learning_rate=0.0005) #decrease learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with class weights
history = model.fit(X_train_padded, y_train_binary, epochs=20, batch_size=64, validation_split=0.15, callbacks=[early_stopping], class_weight=class_weights_dict) #epochs from 10, batch size from 64 and validation from 0.1

# Evaluate the model on the test set
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_binary, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test_binary, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate F1 score
f1score = f1_score(y_test_binary, y_pred)
print(f"F1 Score: {f1score:.4f}")

Epoch 1/20
6098/6098 [==============================] - 1807s 296ms/step - loss: 0.6587 - accuracy: 0.6001 - val_loss: 0.6413 - val_accuracy: 0.6209
Epoch 2/20
6098/6098 [==============================] - 1655s 271ms/step - loss: 0.6360 - accuracy: 0.6305 - val_loss: 0.6309 - val_accuracy: 0.6329
Epoch 3/20
6098/6098 [==============================] - 1663s 273ms/step - loss: 0.6112 - accuracy: 0.6561 - val_loss: 0.6573 - val_accuracy: 0.6001
Epoch 4/20
6098/6098 [==============================] - 1635s 268ms/step - loss: 0.5827 - accuracy: 0.6800 - val_loss: 0.6758 - val_accuracy: 0.5990
Epoch 5/20
3587/3587 [==============================] - 256s 71ms/step
Test Accuracy: 0.6327
Confusion Matrix:
[[57002 27647]
 [14508 15625]]
F1 Score: 0.4257


In [12]:
#fine tuning the hyperparameters of the models
#Improving accuracy for lstm after addressing class imbalance
# Build the Bidirectional LSTM model
embedding_dim = 256 #64 -> 128
lstm_units = 256 #128 -> 256

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(units=lstm_units)))
model.add(Dropout(0.4)) #0.5 -> 0.4
model.add(Dense(1, activation='sigmoid'))

# Compile the model with class weights
optimizer = Adam(learning_rate=0.0005) #decrease learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with class weights
history = model.fit(X_train_padded, y_train_binary, epochs=20, batch_size=64, validation_split=0.15, callbacks=[early_stopping], class_weight=class_weights_dict) #epochs from 10, batch size from 64 and validation from 0.1

# Evaluate the model on the test set
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_binary, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test_binary, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate F1 score
f1score = f1_score(y_test_binary, y_pred)
print(f"F1 Score: {f1score:.4f}")

Epoch 1/20
6098/6098 [==============================] - 2345s 384ms/step - loss: 0.6584 - accuracy: 0.5998 - val_loss: 0.6348 - val_accuracy: 0.6478
Epoch 2/20
6098/6098 [==============================] - 2275s 373ms/step - loss: 0.6340 - accuracy: 0.6317 - val_loss: 0.6466 - val_accuracy: 0.6215
Epoch 3/20
6098/6098 [==============================] - 2358s 387ms/step - loss: 0.6039 - accuracy: 0.6614 - val_loss: 0.6727 - val_accuracy: 0.5712
Epoch 4/20
3587/3587 [==============================] - 288s 80ms/step
Test Accuracy: 0.6485
Confusion Matrix:
[[59731 24918]
 [15427 14706]]
F1 Score: 0.4216


In [7]:
#fine tuning the hyperparameters of the models by adding additional lstm layer
#Improving accuracy for lstm after addressing class imbalance
# Build the Bidirectional LSTM model
embedding_dim = 256 #64 -> 128
lstm_units = 256 #128 -> 256

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(units=lstm_units,return_sequences=True)))
model.add(Bidirectional(LSTM(units=lstm_units))) #additional layer
model.add(Dropout(0.4)) #0.5 -> 0.4
model.add(Dense(1, activation='sigmoid'))

# Compile the model with class weights
optimizer = Adam(learning_rate=0.0005) #decrease learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with class weights
history = model.fit(X_train_padded, y_train_binary, epochs=20, batch_size=64, validation_split=0.15, callbacks=[early_stopping], class_weight=class_weights_dict) #epochs from 10, batch size from 64 and validation from 0.1

# Evaluate the model on the test set
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_binary, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test_binary, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate F1 score
f1score = f1_score(y_test_binary, y_pred)
print(f"F1 Score: {f1score:.4f}")

Epoch 1/20
6098/6098 [==============================] - 8045s 1s/step - loss: 0.6594 - accuracy: 0.5990 - val_loss: 0.6339 - val_accuracy: 0.6333
Epoch 2/20
6098/6098 [==============================] - 7758s 1s/step - loss: 0.6362 - accuracy: 0.6295 - val_loss: 0.6315 - val_accuracy: 0.6373
Epoch 3/20
6098/6098 [==============================] - 7997s 1s/step - loss: 0.6084 - accuracy: 0.6579 - val_loss: 0.6457 - val_accuracy: 0.6143
Epoch 4/20
6098/6098 [==============================] - 7858s 1s/step - loss: 0.5727 - accuracy: 0.6884 - val_loss: 0.6759 - val_accuracy: 0.6069
Epoch 5/20
3587/3587 [==============================] - 625s 174ms/step
Test Accuracy: 0.6385
Confusion Matrix:
[[58118 26531]
 [14967 15166]]
F1 Score: 0.4223


Additional layers can lead to overfitting, especially if the model is too complex for the given dataset. Overfitting occurs when the model learns the training data too well, including its noise and outliers, and fails to generalize to new, unseen data.

In [11]:
#fine tuning the hyperparameters of the models with regularisation
#Improving accuracy for lstm after addressing class imbalance
# Build the Bidirectional LSTM model
embedding_dim = 256 #64 -> 128
lstm_units = 256 #128 -> 256

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(units=lstm_units, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))) #additional layer + regularisation
model.add(Dropout(0.6)) #0.5 -> 0.6
model.add(Dense(1, activation='sigmoid'))

# Compile the model with class weights
optimizer = Adam(learning_rate=0.0005) #decrease learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with class weights
history = model.fit(X_train_padded, y_train_binary, epochs=20, batch_size=64, validation_split=0.15, callbacks=[early_stopping], class_weight=class_weights_dict) #epochs from 10, batch size from 64 and validation from 0.1

# Evaluate the model on the test set
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_binary, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test_binary, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate F1 score
f1score = f1_score(y_test_binary, y_pred)
print(f"F1 Score: {f1score:.4f}")

Epoch 1/20
6098/6098 [==============================] - 4689s 768ms/step - loss: 0.7617 - accuracy: 0.5725 - val_loss: 0.6632 - val_accuracy: 0.5989
Epoch 2/20
6098/6098 [==============================] - 4418s 725ms/step - loss: 0.6571 - accuracy: 0.6147 - val_loss: 0.6808 - val_accuracy: 0.5785
Epoch 3/20
6098/6098 [==============================] - 4305s 706ms/step - loss: 0.6499 - accuracy: 0.6267 - val_loss: 0.6951 - val_accuracy: 0.5426
Epoch 4/20
6098/6098 [==============================] - 4189s 687ms/step - loss: 0.6451 - accuracy: 0.6334 - val_loss: 0.6542 - val_accuracy: 0.6289
Epoch 5/20
6098/6098 [==============================] - 4382s 719ms/step - loss: 0.6408 - accuracy: 0.6402 - val_loss: 0.6758 - val_accuracy: 0.5920
Epoch 6/20
6098/6098 [==============================] - 4178s 685ms/step - loss: 0.6372 - accuracy: 0.6443 - val_loss: 0.6540 - val_accuracy: 0.6198
Epoch 7/20
6098/6098 [==============================] - 4130s 677ms/step - loss: 0.6341 - accuracy: 0.6481

The strength of the regularization may be too high, causing the model to penalize the weights too much

In [17]:
#fine tuning the hyperparameters of the models with regularisation
#Improving accuracy for lstm after addressing class imbalance
# Build the Bidirectional LSTM model
embedding_dim = 256 #64 -> 128
lstm_units = 256 #128 -> 256

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(units=lstm_units, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)))) #additional layer + regularisation adjusted
model.add(Dropout(0.6)) #0.5 -> 0.6
model.add(Dense(1, activation='sigmoid'))

# Compile the model with class weights
optimizer = Adam(learning_rate=0.0005) #decrease learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with class weights
history = model.fit(X_train_padded, y_train_binary, epochs=20, batch_size=64, validation_split=0.15, callbacks=[early_stopping], class_weight=class_weights_dict) #epochs from 10, batch size from 64 and validation from 0.1

# Evaluate the model on the test set
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_binary, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test_binary, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate F1 score
f1score = f1_score(y_test_binary, y_pred)
print(f"F1 Score: {f1score:.4f}")

Epoch 1/20
6098/6098 [==============================] - 5889s 965ms/step - loss: 0.6768 - accuracy: 0.5861 - val_loss: 0.6712 - val_accuracy: 0.5802
Epoch 2/20
6098/6098 [==============================] - 3911s 641ms/step - loss: 0.6509 - accuracy: 0.6204 - val_loss: 0.6418 - val_accuracy: 0.6335
Epoch 3/20
6098/6098 [==============================] - 22367s 4s/step - loss: 0.6444 - accuracy: 0.6321 - val_loss: 0.6601 - val_accuracy: 0.6068
Epoch 4/20
6098/6098 [==============================] - 4199s 689ms/step - loss: 0.6375 - accuracy: 0.6425 - val_loss: 0.6682 - val_accuracy: 0.5947
Epoch 5/20
3587/3587 [==============================] - 297s 82ms/step
Test Accuracy: 0.6340
Confusion Matrix:
[[57328 27321]
 [14694 15439]]
F1 Score: 0.4236


In [78]:
#bert (1/2)
X = imdb_reviews_bigrams['bigram_text']
y = imdb_reviews_bigrams['is_spoiler']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize and pad sequences
max_len = 50  # You can adjust the maximum sequence length
X_train_tokenized = tokenizer(list(X_train), padding=True, truncation=True, return_tensors='pt', max_length=max_len)
X_test_tokenized = tokenizer(list(X_test), padding=True, truncation=True, return_tensors='pt', max_length=max_len)

# Convert labels to torch tensors
y_train_tensor = torch.tensor(y_train.values)
y_test_tensor = torch.tensor(y_test.values)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
